In [1]:
import numpy as np
import hashlib
import os

from scipy.spatial.transform import Rotation as R
from aist_plusplus.loader import AISTDataset

In [2]:
anno_dir = '/srv/share/datasets/AIST/aist_plusplus_final/'

flag_train = False
flag_val = True
flag_test = True

# create list
seq_names = []
if flag_train:
    seq_names += np.loadtxt(
        os.path.join(anno_dir, "splits/crossmodal_train.txt"), dtype=str
    ).tolist()
if flag_val:
    seq_names += np.loadtxt(
        os.path.join(anno_dir, "splits/crossmodal_val.txt"), dtype=str
    ).tolist()
if flag_test:
    seq_names += np.loadtxt(
        os.path.join(anno_dir, "splits/crossmodal_test.txt"), dtype=str
    ).tolist()

ignore_list = np.loadtxt(
    os.path.join(anno_dir, "ignore_list.txt"), dtype=str
).tolist()
seq_names = [name for name in seq_names if name not in ignore_list]
n_samples = len(seq_names)

seq_names, n_samples

(['gLH_sBM_cAll_d17_mLH4_ch01',
  'gLH_sBM_cAll_d18_mLH4_ch01',
  'gKR_sBM_cAll_d30_mKR2_ch01',
  'gKR_sBM_cAll_d28_mKR2_ch01',
  'gBR_sBM_cAll_d04_mBR0_ch01',
  'gBR_sBM_cAll_d05_mBR0_ch01',
  'gLO_sBM_cAll_d13_mLO2_ch01',
  'gLO_sBM_cAll_d15_mLO2_ch01',
  'gJB_sBM_cAll_d08_mJB5_ch01',
  'gJB_sBM_cAll_d09_mJB5_ch01',
  'gWA_sBM_cAll_d26_mWA0_ch01',
  'gWA_sBM_cAll_d25_mWA0_ch01',
  'gJS_sBM_cAll_d01_mJS3_ch01',
  'gJS_sBM_cAll_d03_mJS3_ch01',
  'gMH_sBM_cAll_d22_mMH3_ch01',
  'gMH_sBM_cAll_d24_mMH3_ch01',
  'gHO_sBM_cAll_d21_mHO5_ch01',
  'gHO_sBM_cAll_d20_mHO5_ch01',
  'gPO_sBM_cAll_d11_mPO1_ch01',
  'gPO_sBM_cAll_d10_mPO1_ch01',
  'gLH_sBM_cAll_d17_mLH4_ch02',
  'gLH_sBM_cAll_d18_mLH4_ch02',
  'gKR_sBM_cAll_d30_mKR2_ch02',
  'gKR_sBM_cAll_d28_mKR2_ch02',
  'gBR_sBM_cAll_d04_mBR0_ch02',
  'gBR_sBM_cAll_d05_mBR0_ch02',
  'gLO_sBM_cAll_d13_mLO2_ch02',
  'gLO_sBM_cAll_d15_mLO2_ch02',
  'gJB_sBM_cAll_d08_mJB5_ch02',
  'gJB_sBM_cAll_d09_mJB5_ch02',
  'gWA_sBM_cAll_d26_mWA0_ch02',
  'gWA_s

In [3]:
def compute_SMPL_motion(seq_name, motion_dir):
    smpl_poses, smpl_scaling, smpl_trans = AISTDataset.load_motion(motion_dir, seq_name)
    smpl_trans /= smpl_scaling
    smpl_poses = R.from_rotvec(
        smpl_poses.reshape(-1, 3)).as_matrix().reshape(smpl_poses.shape[0], -1)
    smpl_motion = np.concatenate([smpl_trans, smpl_poses], axis=-1)
    return smpl_motion


motion_dir = os.path.join(anno_dir, 'motions')
for i, seq_name in enumerate(seq_names[:1]):
    print("processing %d / %d" % (i + 1, n_samples))

    smpl_motion = compute_SMPL_motion(seq_name, motion_dir)
    print(f"Seq name: {seq_name} \t Motion shape: {smpl_motion.shape}")

processing 1 / 40
Seq name: gLH_sBM_cAll_d17_mLH4_ch01 	 Motion shape: (480, 219)


In [4]:
seq_name, smpl_motion.shape

('gLH_sBM_cAll_d17_mLH4_ch01', (480, 219))

In [5]:
h = hashlib.sha1(seq_name.encode('utf-8'))
h_int = int(h.hexdigest(), 16)
seq_name, h, h_int

('gLH_sBM_cAll_d17_mLH4_ch01',
 <sha1 HASH object @ 0x7fa2253fec00>,
 187866566387554190740179474152453353543983648002)

In [6]:
h_str = str(h_int)
h_str

'187866566387554190740179474152453353543983648002'

In [7]:
h_np = np.array([float(h_str[i:i+3]) / 1e3 for i in range(0, len(h_str), 3)]).reshape((1,-1))
h_np, h_np.shape

(array([[0.187, 0.866, 0.566, 0.387, 0.554, 0.19 , 0.74 , 0.179, 0.474,
         0.152, 0.453, 0.353, 0.543, 0.983, 0.648, 0.002]]),
 (1, 16))

In [8]:
target_shape = (256,smpl_motion.shape[-1])
# target_shape = smpl_motion.shape
target_shape

(256, 219)

In [9]:
op_dim = np.prod(target_shape)
op_dim

56064

In [10]:
wt_seed = 101
wt_rng = np.random.default_rng(seed=wt_seed)

hidden_size = 128

w1 = wt_rng.normal(size=(h_np.shape[-1], hidden_size))
b1 = wt_rng.normal(size=hidden_size)
w2 = wt_rng.normal(size=(hidden_size, op_dim))
b2 = wt_rng.normal(size=op_dim)

z1 = h_np @ w1 + b1
op = np.tanh(z1) @ w2 + b2
op.reshape(target_shape).shape

(256, 219)

In [11]:
op

array([[  3.95264557,  18.10119154,   1.37463447, ...,  31.69889754,
        -13.08205681,   2.50961309]])